In [ ]:
import netCDF4
import xarray as xr
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
def get_geotop(y, x, ds,lithok_translation):
    zvalues = ds.z.values.tolist()
    lithok_value = ds.sel(x=x, y=y, method="nearest").values.tolist()
    df_geotop = pd.DataFrame({'bottom':zvalues, 'layer': lithok_value})
    filtered_df = df_geotop[df_geotop["layer"] != df_geotop["layer"].shift(1)].copy()
    filtered_df['top'] = filtered_df['bottom'].shift(-1)
    filtered_df = filtered_df.iloc[::-1].reset_index(drop=True)
    filtered_df = filtered_df.dropna().reset_index(drop=True)
    filtered_df['layer'] = filtered_df['layer'].replace(lithok_translation) # Comment this line if translation of classes is not needed
    return filtered_df

def flatdata(df):
    flattened = df[['top', 'bottom', 'layer']].values.flatten()
    columns = []
    for i in range(1, len(df) + 1):
        columns.extend([f"TOP_L{i}", f"BOT_L{i}", f"Lithoclass_L{i}"])
    result = pd.DataFrame([flattened], columns=columns)
    return result

def add_attributes(result, well, x, y, ID1, z0, z1, z2):
    result.insert(0, 'Location_ID', well)   
    result.insert(1, 'x', x)                
    result.insert(2, 'y', y)                
    result.insert(3, 'ID1', ID1)            
    result.insert(4, 'ID2', ID1.split('_')[0]+'.'+ID1.split('_')[1])            
    result.insert(5, 'surface', z0)         
    result.insert(6, 'top_screen', z1)      
    result.insert(7, 'bot_screen', z2)      
    return result

In [ ]:
url = 'https://www.dinodata.nl/opendap/GeoTOP/geotop.nc'
dataset = xr.open_dataset(url)
file_path = r"C:\Users\fuentesm\Projects\GeoTOP\Data\Input\Geodata\Meetpunten_Grondwateratlas.xlsx"
sheet_name_z = "ipf_plus_ahn_NAP"
df = pd.read_excel(file_path, sheet_name=sheet_name_z, skiprows=10)

df = df.loc[:,:]

lithok = dataset['lithok']
zvalues = dataset.z.values.tolist()

lithok_translation = {
    0: 'antropogeen',
    1: 'organisch materiaal (veen)',
    2: 'klei',
    3: 'klei zandig, leem, kleiig fijn zand',
    5: 'zand fijn',
    6: 'zand matig grof',
    7: 'zand grof',
    8: 'grind',
    9: 'schelpen',
    -127: 'NN',
}

previous_well = None 
list_df = []
for index, row in df.iterrows():
    well = row['PutCode.Filternr'].split('_')[0]
    print(well)
    if well == previous_well:
        print(f"Same well as previous iteration: {well}")
        results = add_attributes(flat_data.copy(), well, row['X_m'], row['Y_m'], row['PutCode.Filternr'], row['z0'], row['z1'], row['z2'])
        list_df.append(results)
    else:
        print(f"New well: {well}")
        df_geotop = get_geotop(row['Y_m'],row['X_m'], lithok, lithok_translation)
        flat_data = flatdata(df_geotop)
        results = add_attributes(flat_data.copy(), well, row['X_m'], row['Y_m'], row['PutCode.Filternr'], row['z0'], row['z1'], row['z2'])
        list_df.append(results)
    previous_well = well

B49D0097
New well: B49D0097
B49D0097
Same well as previous iteration: B49D0097
B49D0097
Same well as previous iteration: B49D0097
B31E0177
New well: B31E0177
B31E0177
Same well as previous iteration: B31E0177
B31E0177
Same well as previous iteration: B31E0177
B38F0527
New well: B38F0527
B38F0527
Same well as previous iteration: B38F0527
B38F0527
Same well as previous iteration: B38F0527
B26C0434
New well: B26C0434
B26C0434
Same well as previous iteration: B26C0434
B26A0244
New well: B26A0244
B26A0244
Same well as previous iteration: B26A0244
B51G0465
New well: B51G0465
B51G0465
Same well as previous iteration: B51G0465
B15G0054
New well: B15G0054
B15G0054
Same well as previous iteration: B15G0054
B15G0054
Same well as previous iteration: B15G0054
B15H0031
New well: B15H0031
B15H0031
Same well as previous iteration: B15H0031
B16C0024
New well: B16C0024
B16C0024
Same well as previous iteration: B16C0024
B16C0024
Same well as previous iteration: B16C0024
B58D0388
New well: B58D0388
B58D03

In [55]:
geotop_df = pd.concat(list_df, ignore_index=True)
geotop_df.to_csv("geotop_translated.csv", index=False)
geotop_df.to_excel("geotop_translated.xlsx", index=False)
geotop_df.iloc[:,:12]

,Location_ID,x,y,ID1,ID2,surface,top_screen,bot_screen,TOP_L1,BOT_L1,Lithoclass_L1,TOP_L2
0,B49D0097,76960.00,382580.00,B49D0097_1,B49D0097.1,1.50,-2.21,-3.21,1.0,-3.5,zand fijn,-3.5
1,B49D0097,76960.00,382580.00,B49D0097_2,B49D0097.2,1.50,-12.13,-13.13,1.0,-3.5,zand fijn,-3.5
2,B49D0097,76960.00,382580.00,B49D0097_3,B49D0097.3,1.50,-22.63,-23.63,1.0,-3.5,zand fijn,-3.5
3,B31E0177,124375.00,468025.00,B31E0177_1,B31E0177.1,-1.90,-9.90,-11.90,-2.0,-8.5,organisch materiaal (veen),-8.5
4,B31E0177,124375.00,468025.00,B31E0177_2,B31E0177.2,-1.90,-14.90,-16.90,-2.0,-8.5,organisch materiaal (veen),-8.5
...,...,...,...,...,...,...,...,...,...,...,...,...
2566,28FP0097,251666.48,497087.12,28FP0097_2,28FP0097.2,30.36,7.37,5.37,NaN,NaN,NaN,NaN
2567,28FP0102,252023.24,496905.93,28FP0102_1,28FP0102.1,31.90,23.90,22.90,NaN,NaN,NaN,NaN
2568,28FP0102,252023.24,496905.93,28FP0102_2,28FP0102.2,31.90,9.41,7.41,NaN,NaN,NaN,NaN
2569,28FP0105,252885.26,496595.10,28FP0105_1,28FP0105.1,36.38,28.41,26.41,NaN,NaN,NaN,NaN
